In [1]:
import pandas as pd

In [2]:
features_data = pd.read_csv('features_data.csv')
features_data.head()

,class,fold,salience,amplitude_envelope,root_mean_square,zero_crossing_rate,mfcc,spectral_centroid,bandwidth
0,dog_bark,5,1,2.501107,0.859733,0.701485,1114.7451,11002.153867,9055.166227
1,children_playing,5,1,0.207636,0.076948,2.715422,5759.2744,38984.715069,35987.449939
2,children_playing,5,1,0.141984,0.056998,1.999477,6220.2430,33621.214468,35413.731359
3,children_playing,5,1,0.254567,0.092671,2.646289,5578.5050,41672.174391,41306.245923
4,children_playing,5,1,0.154707,0.060585,2.300883,6048.3857,36180.503459,35524.610921


## Klasyfikator KNN

In [3]:
class KNN:
    @staticmethod
    def minkowski(v1, v2, m):
        tmp = 0
        for i in range(2, len(v1) - 1):
            tmp += abs(v1[i] - v2[i]) ** m
        return tmp ** (1 / m)

    @staticmethod
    def quick_sort(t: pd.DataFrame, d: list, l: int, r: int):
        if l < r:
            pivot = int((l + r) / 2)
            d[pivot], d[r] = d[r], d[pivot]
            t.iloc[pivot], t.iloc[r] = t.iloc[r], t.iloc[pivot]
            j = l
            for i in range(l, r):
                if d[i] <= d[r]:
                    d[i], d[j] = d[j], d[i]
                    t.iloc[i], t.iloc[j] = t.iloc[j], t.iloc[i]
                    j += 1
            d[j], d[r] = d[r], d[j]
            t.iloc[j], t.iloc[r] = t.iloc[r], t.iloc[j]
            KNN.quick_sort(t, d, l, j - 1)
            KNN.quick_sort(t, d, j + 1, r)
        
    @staticmethod
    def clustering(x, sample, k, m):
        distances = []
        for i in range(0, len(x)):
            distances.append(KNN.minkowski(x.iloc[i], sample, m))
            
        KNN.quick_sort(x, distances, 0, len(x) - 1)
            
        classes = {c : 0 for c in x['class']}
        for i in range(0, k):
            classes[x.loc[i, 'class']] += 1
            
        return max(classes, key=classes.get)

    @staticmethod
    def normalize(x):
        columns = x.iloc[:, 2:]
        for column in columns:
            data = x.loc[:, column]
            minimum = min(data)
            maximum = max(data)
            for i in range(0, len(x)):
                x.at[i, column] = (x.at[i, column] - minimum) / (maximum - minimum)
        return x

In [4]:
features_data = KNN.normalize(features_data)
features_data.head()

,class,fold,salience,amplitude_envelope,root_mean_square,zero_crossing_rate,mfcc,spectral_centroid,bandwidth
0,dog_bark,5,0,0.132132,0.079255,0.046789,0.089087,0.063560,0.078529
1,children_playing,5,0,0.010540,0.006809,0.190093,0.551544,0.295723,0.430702
2,children_playing,5,0,0.007059,0.004963,0.139149,0.597443,0.251223,0.423200
3,children_playing,5,0,0.013028,0.008264,0.185174,0.533545,0.318020,0.500252
4,children_playing,5,0,0.007734,0.005295,0.160596,0.580331,0.272457,0.424650


## Sprawdzanie dokładności klasyfikatora

In [ ]:
corrected = 0

for i in range(1, 11):
    train = features_data[features_data['fold'] != i]
    test = features_data[features_data['fold'] == i]
    
    for j in range(0, len(features_data)):
        if features_data.loc[j, 'class'] == KNN.clustering(train, test.iloc[j], 10, 2):
            corrected += 1

accuracy = corrected / 10 * len(features_data) * 100

In [ ]:
accuracy